In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from thefuzz import fuzz, process
import missingno as msno
from salmonella_study.config import Config
from salmonella_study import data_processing as processing
from salmonella_study import utils

# Matched counties


In [4]:
matched_counties = pd.read_csv("c:/Users/Zayan/Documents/code/personal_repos/salmonella/Data/processed_data/matched_counties.csv")

In [5]:
matched_counties.head()

,Unnamed: 0,Census,Matched SVI County,Matched PulseNet County,Matched MMG County
0,0,"Autauga County, Alabama",Autauga,COUNTY,"Autauga County, Alabama"
1,1,"Baldwin County, Alabama",Baldwin,Baldwin,"Baldwin County, Alabama"
2,2,"Barbour County, Alabama",Barbour,COUNTY,"Barbour County, Alabama"
3,3,"Bibb County, Alabama",Bibb,Bibb,"Bibb County, Alabama"
4,4,"Blount County, Alabama",Blount,Blount,"Blount County, Alabama"


In [6]:
# Import PulseNet data
pulsenet_path = r"C:\Users\Zayan\Documents\code\personal_repos\salmonella\Data\raw_data\PulseNet data\Export_2020-2023.xlsx"
pulsenet_data = pd.read_excel(pulsenet_path, sheet_name="2020-2023")

# Import Map the Meal Gap (MMG) data
mmg_path = r"c:/Users/Zayan/Documents/code/personal_repos/salmonella/Data/raw_data/MMG/MMG2022_2020-2019Data_ToShare.xlsx"
mmg_data = pd.read_excel(mmg_path, sheet_name='County')

# Import Social Vulnerability Index (SVI) data
svi_path = r"C:\Users\Zayan\Documents\code\personal_repos\salmonella\Data\raw_data\social_vulnerability_index\SVI_2020_US_county.csv"
svi_data = pd.read_csv(svi_path)

# Display the first few rows of each dataset
print("PulseNet Data:")
print(pulsenet_data.head())
print("\nMap the Meal Gap Data:")
print(mmg_data.head())
print("\nSocial Vulnerability Index Data:")
print(svi_data.head())



PulseNet Data:
                Key AntigenForm  AntigenForm_wgs           IsolatDate LabID  \
0  FL___JBS19003284         NaN  I 7:e,h:e,n,z15  2019-11-21 00:00:00    FL   
1  FL___JBS19003283         NaN        I 9:g,m:-  2019-11-25 00:00:00    FL   
2  FL___JBS19003285         NaN        I 4:i:1,2  2019-11-29 00:00:00    FL   
3  FL___JBS19003286         NaN      I 4:e,h:1,2  2019-11-30 00:00:00    FL   
4     MA___19EN7649         NaN        I 7:r:1,5  2019-12-16 00:00:00    MA   

  Outbreak  PatientAgeDays PatientAgeMonths PatientAgeYears PatientSex  ...  \
0      NaN             NaN              NaN              19     FEMALE  ...   
1      NaN             NaN              NaN               4       MALE  ...   
2      NaN             NaN                2             NaN       MALE  ...   
3      NaN             NaN                4             NaN       MALE  ...   
4      NaN             NaN              NaN              26       MALE  ...   

  Serotype_wgs   SourceCity SourceC

In [7]:
pulsenet_data.loc[pulsenet_data['SourceCounty'] == 'COUNTY']


,Key,AntigenForm,AntigenForm_wgs,IsolatDate,LabID,Outbreak,PatientAgeDays,PatientAgeMonths,PatientAgeYears,PatientSex,...,Serotype_wgs,SourceCity,SourceCountry,SourceCounty,SourceSite,SourceState,SourceType,TypeDetails,WGS_id,Unnamed: 21
35523,SC___SALM21-0028,NaN,"I 8:e,h:1,2",2021-01-05 00:00:00,SC,NaN,NaN,NaN,14,FEMALE,...,Newport,NaN,USA,COUNTY,Urine,SC,Human,NaN,PNUSAS190864,NaN


In [2]:
pulsenet_data.loc[pulsenet_data['SourceCounty'] == 'COUNTY']


NameError: name 'pulsenet_data' is not defined

Verify that counties marked as 'COUNTY' in matched_counties['Matched PulseNet County'] 
are indeed not present in the PulseNet data

In [9]:
# Function to find similar county names in PulseNet data
def find_similar_counties(df, county_name, similarity_threshold=80):
    matches = process.extract(county_name, df['SourceCounty'], limit=None)
    return [match[0] for match in matches if match[1] >= similarity_threshold]

# Filter matched_counties for rows where 'Matched PulseNet County' is 'COUNTY'
counties_to_check = matched_counties['Census'][((matched_counties['Matched PulseNet County'] == 'COUNTY') | (matched_counties['Matched PulseNet County'] == ''))]

# Verify each county
for index, row in counties_to_check.iterrows():
    census_county = row['Census Matched SVI County']
    similar_counties = find_similar_counties(pulsenet_data, census_county)
    
    if similar_counties:
        print(f"Warning: {census_county} marked as 'COUNTY' but similar names found in PulseNet: {similar_counties}")
    else:
        print(f"Verified: {census_county} not found in PulseNet data")

# Count and display the number of verified and unverified counties
verified_count = counties_to_check.shape[0] - sum(1 for _ in counties_to_check.iterrows() if find_similar_counties(pulsenet_data, _[1]['Census Matched SVI County']))
unverified_count = counties_to_check.shape[0] - verified_count

print(f"\nSummary:")
print(f"Total counties checked: {counties_to_check.shape[0]}")
print(f"Verified (not found in PulseNet): {verified_count}")
print(f"Unverified (similar names found in PulseNet): {unverified_count}")

AttributeError: 'Series' object has no attribute 'iterrows'

In [15]:
def find_specific_counties(df, county_name, similarity_threshold=80):
    matches = process.extract(county_name, df['SourceCounty'], limit=None)
    return [match[0] for match in matches if match[1] >= similarity_threshold]

find_specific_counties(pulsenet_data, 'kusilvak')
find_specific_counties(pulsenet_data, 'autauga')



['Watauga', 'Watauga', 'WATAUGA', 'WATAUGA', 'WATAUGA', 'WATAUGA']

# SVI+MMG+PulseNet merged

In [3]:
processed_data_path = Config.PROCESSED_DATA_DIR

In [5]:
df = pd.read_csv(os.path.join(processed_data_path, 'salmonella_population', 'sense-d_socioecono_salmonella_MO_2020.csv'))

In [6]:
df

,STCNTY,County,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV150,E_UNEMP,...,% FI > High Threshold,Child Food Insecurity Rate (1 Year),# of Food Insecure Children (1 Year),% food insecure children in HH w/ HH incomes below 185 FPL,% food insecure children in HH w/ HH incomes above 185 FPL,Cost Per Meal (1 Year),Weighted weekly $ needed by FI,Weighted Annual Food Budget Shortfall,salmonella_count,salmonella per 100000
0,29001,Adair,29001,"Adair County, Missouri",567.323630,25468,11595,9078,7796,649,...,0.234,0.129,610,0.89,0.11,2.94,15.582552,1626000,4.0,15.830299
1,29003,Andrew,29003,"Andrew County, Missouri",432.631321,17554,7337,6799,2867,354,...,0.389,0.088,360,0.94,0.06,2.97,15.774208,789000,3.0,16.571839
2,29005,Atchison,29005,"Atchison County, Missouri",547.285156,5180,2947,2549,1200,92,...,0.256,0.140,140,0.91,0.09,3.23,17.148613,323000,NaN,NaN
3,29007,Audrain,29007,"Audrain County, Missouri",692.249287,25336,10909,9349,6926,583,...,0.266,0.182,1060,0.98,0.02,2.89,15.335352,1623000,3.0,12.103607
4,29009,Barry,29009,"Barry County, Missouri",778.121661,35615,17656,13872,10171,514,...,0.252,0.191,1540,0.91,0.09,2.96,15.728063,2467000,6.0,17.369152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,29221,Washington,29221,"Washington County, Missouri",759.865711,24819,11080,9278,7828,710,...,0.195,0.211,1220,0.87,0.13,2.83,15.014567,1995000,1.0,4.269126
111,29223,Wayne,29223,"Wayne County, Missouri",759.185172,13058,8117,5438,4939,378,...,0.221,0.248,650,0.84,0.16,2.79,14.788149,1180000,3.0,27.432334
112,29225,Webster,29225,"Webster County, Missouri",592.562929,39127,14821,13697,10504,859,...,0.308,0.143,1510,0.95,0.05,3.02,16.006894,2442000,8.0,20.505985
113,29227,Worth,29227,"Worth County, Missouri",266.628521,2001,1270,808,479,19,...,0.398,0.172,70,0.80,0.20,3.23,17.169329,135000,NaN,NaN


In [8]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 129 columns):
 #    Column                                                      Dtype  
---   ------                                                      -----  
 0    STCNTY                                                      int64  
 1    County                                                      object 
 2    FIPS                                                        int64  
 3    LOCATION                                                    object 
 4    AREA_SQMI                                                   float64
 5    E_TOTPOP                                                    int64  
 6    E_HU                                                        int64  
 7    E_HH                                                        int64  
 8    E_POV150                                                    int64  
 9    E_UNEMP                                                     int64  
 10   

In [9]:
col_pos = {'County normalized':2,'2020 population':4, 'State':3, 'Year':5}
df = processing.rearrange_cols(df, col_pos)

In [10]:
df.head()

,STCNTY,County,County normalized,State,FIPS,Year,2020 population,LOCATION,AREA_SQMI,E_TOTPOP,...,% FI > High Threshold,Child Food Insecurity Rate (1 Year),# of Food Insecure Children (1 Year),% food insecure children in HH w/ HH incomes below 185 FPL,% food insecure children in HH w/ HH incomes above 185 FPL,Cost Per Meal (1 Year),Weighted weekly $ needed by FI,Weighted Annual Food Budget Shortfall,salmonella_count,salmonella per 100000
0,29001,Adair,Adair,MO,29001,2020,25268,"Adair County, Missouri",567.323630,25468,...,0.234,0.129,610,0.89,0.11,2.94,15.582552,1626000,4.0,15.830299
1,29003,Andrew,Andrew,MO,29003,2020,18103,"Andrew County, Missouri",432.631321,17554,...,0.389,0.088,360,0.94,0.06,2.97,15.774208,789000,3.0,16.571839
2,29005,Atchison,Atchison,MO,29005,2020,5306,"Atchison County, Missouri",547.285156,5180,...,0.256,0.140,140,0.91,0.09,3.23,17.148613,323000,NaN,NaN
3,29007,Audrain,Audrain,MO,29007,2020,24786,"Audrain County, Missouri",692.249287,25336,...,0.266,0.182,1060,0.98,0.02,2.89,15.335352,1623000,3.0,12.103607
4,29009,Barry,Barry,MO,29009,2020,34544,"Barry County, Missouri",778.121661,35615,...,0.252,0.191,1540,0.91,0.09,2.96,15.728063,2467000,6.0,17.369152


In [13]:
for i in df.iterrows():
    print(i[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
